In [1]:
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


In [2]:
data = np.load('/Users/harshitgupta/Desktop/vs /VS-Data-Den/data_cleaning/sales_matrix.npy')

In [3]:
regional_masters = data[1:, 0]
dates = data[0, 1:]


In [4]:
endog = data[1:, 1:].astype(float) 

In [5]:
train_size = int(len(dates) * 0.8) 
train_dates = dates[:train_size]
test_dates = dates[train_size:]

In [6]:
mape_values = []

In [8]:
for i, regional_master in enumerate(regional_masters):
    # Split the data for this regional_master
    train_data = endog[i, :train_size]
    test_data = endog[i, train_size:]

    # Fit the VAR model
    model = sm.tsa.VAR([train_data])
    model_fitted = model.fit()

    # Make forecasts
    forecast_period = len(test_data)
    last_train_value = train_data[-model_fitted.k_ar:]
    forecasts = model_fitted.forecast(last_train_value, steps=forecast_period)

    # Evaluate the forecasts using MAPE
    forecasted_values = forecasts.reshape(-1)
    test_data_non_zero = np.where(test_data == 0, 1e-8, test_data)
    mape = np.mean(np.abs((test_data_non_zero - forecasted_values) / test_data_non_zero)) * 100
    mape_values.append(mape)

    print(f"Regional Master: {regional_master}, MAPE: {mape:.2f}%")

ValueError: zero-size array to reduction operation maximum which has no identity